**Loading of data**

Initial code taken from: https://github.com/pierinim/tutorials/blob/master/GGI_Jan2021/Lecture1/Notebook1_ExploreDataset.ipynb

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [2]:
A = ! ls tutorials/Data/JetDataset
fileIN =  f'tutorials/Data/JetDataset/{A[2]}'
print(fileIN)
print(A)

tutorials/Data/JetDataset/jetImage_7_100p_30000_40000.h5
['jetImage_7_100p_0_10000.h5', 'jetImage_7_100p_10000_20000.h5', 'jetImage_7_100p_30000_40000.h5', 'jetImage_7_100p_40000_50000.h5', 'jetImage_7_100p_50000_60000.h5', 'jetImage_7_100p_60000_70000.h5', 'jetImage_7_100p_70000_80000.h5', 'jetImage_7_100p_80000_90000.h5']


In [3]:
file = h5py.File(fileIN)

In [4]:
print(list(file.keys()))

['jetConstituentList', 'jetFeatureNames', 'jetImage', 'jetImageECAL', 'jetImageHCAL', 'jets', 'particleFeatureNames']


In [5]:
jetconstituents = file.get('jetConstituentList')


# Shape: num_jets x num_particles x 4
fourvectors = jetconstituents[:, :, :4] # Particles (px, py, pz, E)


In [6]:
# Creating full matrix of inner products
M = np.diag([-1, -1, -1, 1])
inner_prods = np.einsum("npi, ij, nqj->npq", fourvectors, M, fourvectors)
print(inner_prods.shape)

(10000, 100, 100)


In [7]:
jet_data = np.array(file.get('jets'))
target = jet_data[:, -6:-1]
data = np.expand_dims(inner_prods, axis=-1)
print(f'Target shape: {target.shape} Data shape: {data.shape}')


Target shape: (10000, 5) Data shape: (10000, 100, 100, 1)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33)

In [9]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6700, 100, 100, 1) (3300, 100, 100, 1) (6700, 5) (3300, 5)


**Using the PELICAN architecture**

In [10]:
import tensorflow as tf
from keras.models import Sequential
from layers import Msg, LinEq2v2, LinEq2v0
from keras.layers import Dropout, Dense, Flatten
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
from keras.optimizers import Adam

from tqdm.keras import TqdmCallback


c:\Users\otkul\ETH\vår\project\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
dropout_rate = 0.02
model = Sequential(layers= [
    Msg(30, activation='sigmoid'),
    LinEq2v2(40, activation='sigmoid'),
    LinEq2v0(80, activation='sigmoid'),
    Dense(5, activation='softmax')
]
)


model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseCategoricalAccuracy(), 'accuracy'],
)



In [12]:
EPOCHS = 10
BATCH = 128
# model.build((BATCH, 100, 100, 1))



In [13]:
y = model(data[0])
model.summary()


ValueError: Exception encountered when calling layer 'sequential' (type Sequential).

Input 0 of layer "dense" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (80,)

Call arguments received by layer 'sequential' (type Sequential):
  • inputs=tf.Tensor(shape=(100, 100, 1), dtype=float64)
  • training=None
  • mask=None

In [ ]:
print(y[2])


tf.Tensor([0.22318168 0.37661195 0.22950271 0.00555173 0.1651519 ], shape=(5,), dtype=float32)


In [ ]:

history = model.fit(
    X_train,
    y_train,
    epochs = EPOCHS,
    batch_size = BATCH,
    validation_data=(X_test, y_test),
    callbacks=[TqdmCallback(verbose=1)],
    verbose=0
)


  0%|          | 0/10 [00:00<?, ?epoch/s]

ValueError: in user code:

    File "c:\Users\otkul\ETH\vår\project\.env\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\otkul\ETH\vår\project\.env\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\otkul\ETH\vår\project\.env\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\otkul\ETH\vår\project\.env\Lib\site-packages\keras\src\engine\training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\otkul\ETH\vår\project\.env\Lib\site-packages\keras\src\engine\training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "c:\Users\otkul\ETH\vår\project\.env\Lib\site-packa